In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("processed.csv")
df

,Unnamed: 0,clean_comment,category
0,0,family mormon never tried explain still stare ...,1
1,1,buddhism much lot compatible christianity espe...,1
2,2,seriously say thing first get complex explain ...,-1
3,3,learned want teach different focus goal not wr...,0
4,4,benefit may want read living buddha living chr...,1
...,...,...,...
36241,37243,agree push make nation either pity pakistan in...,-1
36242,37244,jesus,0
36243,37245,kya bhai pure saal chutiya banaya modi aur jab...,1
36244,37246,downvote karna tha par upvote hogaya,0


In [3]:
df.dropna(inplace=True)

In [4]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [5]:
df

,Unnamed: 0,clean_comment,category
0,0,family mormon never tried explain still stare ...,1
1,1,buddhism much lot compatible christianity espe...,1
2,2,seriously say thing first get complex explain ...,-1
3,3,learned want teach different focus goal not wr...,0
4,4,benefit may want read living buddha living chr...,1
...,...,...,...
36241,37243,agree push make nation either pity pakistan in...,-1
36242,37244,jesus,0
36243,37245,kya bhai pure saal chutiya banaya modi aur jab...,1
36244,37246,downvote karna tha par upvote hogaya,0


In [6]:
df['category'].value_counts()

category
 1    15666
 0    12377
-1     8200
Name: count, dtype: int64

Distribution of categories in the original dataset is like this

In [7]:
df['category'].value_counts(normalize=True) * 100

category
 1    43.224899
 0    34.150043
-1    22.625059
Name: proportion, dtype: float64

In [8]:
# Step 4: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

df['clean_comment']: This column contains the cleaned text comments (or input features) from the DataFrame df.
df['category']: This column represents the target variable or labels for classification.

stratify=df['category']: Stratification helps maintain the same distribution of categories in both the training and testing sets as in the original dataset. This is especially important if the dataset is imbalanced, ensuring that each category is proportionately represented in both sets.


In [9]:
X_train

27844    get point internet anonymity brings worst peop...
14784    bjp kicked even low level chutiyas getting muc...
6502                                     gay psychological
20472                                  thought gonna blank
25884                              british making film bjp
                               ...                        
29005                gundas chutiyas but terrorist stretch
35464                                           thurki cry
23933    dude gaming related dinner event mma fighter b...
7316                                                 boner
34557    tbh growing consumption led growth though impr...
Name: clean_comment, Length: 28994, dtype: object

In [10]:
y_train.value_counts()

category
 1    12533
 0     9901
-1     6560
Name: count, dtype: int64

In [11]:
y_train.value_counts(normalize=True) * 100

category
 1    43.226185
 0    34.148445
-1    22.625371
Name: proportion, dtype: float64

The distribution is almost exactly same.

In [12]:
y_test.value_counts(normalize=True) * 100

category
 1    43.219754
 0    34.156435
-1    22.623810
Name: proportion, dtype: float64

In [13]:
ngram_range = (1, 3)  # Trigram setting (from 1 to 3 means, it will create combination of unigrams, bigrams and trigrams)
max_features = 1000  # Set max_features to 1000 for TF-IDF

# Step 2: Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)


In [14]:
vectorizer

TfidfVectorizer(max_features=1000, ngram_range=(1, 3))

In [15]:
X_train_vec = vectorizer.fit_transform(X_train)  # Fit only on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

In [16]:
X_train

27844    get point internet anonymity brings worst peop...
14784    bjp kicked even low level chutiyas getting muc...
6502                                     gay psychological
20472                                  thought gonna blank
25884                              british making film bjp
                               ...                        
29005                gundas chutiyas but terrorist stretch
35464                                           thurki cry
23933    dude gaming related dinner event mma fighter b...
7316                                                 boner
34557    tbh growing consumption led growth though impr...
Name: clean_comment, Length: 28994, dtype: object

In [17]:
X_train_vec

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 280222 stored elements and shape (28994, 1000)>

In [18]:
dense_matrix = X_train_vec.toarray()
print(dense_matrix)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [19]:
X_train_vec[0].toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.08260609, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.07837933, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [20]:
X_train_vec[0].toarray().flatten()


array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.08260609, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.07837933, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [21]:
## Let us count how many numbers are non zero in the vector representation
cnt = 0
for i in X_train_vec[0].toarray().flatten():
    if i > 0:
        cnt += 1
    
print(cnt)

64


In [22]:
len(X_train[0].split())

26

Yes, in the context of n-grams, specifying `n_grams = (1, 3)` means that you want to generate n-grams of sizes ranging from 1 to 3. This includes:

- **1-grams** (unigrams): Single words
- **2-grams** (bigrams): Pairs of consecutive words
- **3-grams** (trigrams): Triples of consecutive words

For example, if your input text is "I love data science", the n-grams would be:

- **1-grams**: ["I", "love", "data", "science"]
- **2-grams**: ["I love", "love data", "data science"]
- **3-grams**: ["I love data", "love data science"]

This approach allows you to capture different levels of context and relationships between words in your text data.

The setting `max_features=1000` in `TfidfVectorizer` means it will consider only the **top 1,000 words** from the vocabulary. Here’s how it works:

- `TfidfVectorizer` builds a vocabulary of all unique words in the dataset.
- It calculates the **Term Frequency-Inverse Document Frequency (TF-IDF)** for each word in each document.
- When `max_features=1000` is specified, it **limits the vocabulary** to the top 1,000 words based on their importance. By default, it will rank words by their TF-IDF score, though this ranking can also be based on other metrics, like term frequency.

This setting is helpful for:
1. **Reducing dimensionality**: Only the most important 1,000 words are considered, which reduces the size of the feature matrix.
2. **Improving model performance**: By focusing on the most relevant words, the model can often generalize better and train faster.

If you want to control which words are selected, you can use additional parameters like `max_df`, `min_df`, or even specify custom stop words to filter out certain terms.

When `TfidfVectorizer` is limited to the top `max_features` (e.g., 1,000 words), any words that are not among those selected—**out-of-vocabulary (OOV)** words—are simply ignored during transformation. Here’s how it handles those OOV words:

1. **During Fitting (`fit` or `fit_transform`)**:
   - `TfidfVectorizer` builds a vocabulary of only the top words according to the `max_features` criteria.
   - Any word that doesn’t fall into this top set of 1,000 is excluded from the vocabulary and is not assigned a TF-IDF score.

2. **During Transformation (`transform`)**:
   - When you call `.transform()` on new or test data, only the words in the vocabulary are considered.
   - Any words not in the vocabulary are ignored, meaning they do not contribute to the generated feature vectors.

For example, if you have `max_features=1000` and a word like "banana" isn't in the top 1,000 words, then "banana" will have no effect on the vectorized representation of any document containing it. Essentially, it’s as if the OOV words do not exist in the documents from the perspective of the vectorizer.

This approach is typical when using bag-of-words or TF-IDF methods, as the vocabulary is fixed based on training data and ignores any unseen words or less relevant words during transformation.

The "top 1000 features" in the context of n-grams, it typically means you are looking to select the most relevant or frequent features from the entire mixture of n-grams generated. In this case, the top 1000 features would be a combination of:

- **1-grams**: The most frequent single words.
- **2-grams**: The most common pairs of words.
- **3-grams**: The most common triplets of words.

The selection of the top 1000 features could be based on their frequency of occurrence across your dataset, their importance in relation to the task at hand (like classification or regression), or other criteria like term frequency-inverse document frequency (TF-IDF) scores.

This combined approach helps capture a broader context and semantic relationships in the text data, enhancing the performance of machine learning models, especially in natural language processing tasks.

In [23]:
# Get feature names
feature_names = vectorizer.get_feature_names_out()


# View the top 1000 words
top_words = feature_names[:1000]
print(top_words)

['000' '100' '2014' '2016' '2018' '2019' '500' 'aap' 'able' 'absolutely'
 'accept' 'according' 'account' 'across' 'act' 'action' 'actor' 'actual'
 'actually' 'add' 'age' 'agenda' 'ago' 'agree' 'ahead' 'air' 'alliance'
 'allowed' 'almost' 'alone' 'along' 'already' 'also' 'alternative'
 'although' 'always' 'amazing' 'america' 'american' 'amit' 'amit shah'
 'among' 'amount' 'android' 'another' 'answer' 'anti' 'anti national'
 'anymore' 'anyone' 'anything' 'anyway' 'app' 'area' 'argument' 'army'
 'around' 'arrested' 'article' 'as' 'ask' 'asked' 'asking' 'assembly'
 'atleast' 'attack' 'attempt' 'attention' 'aur' 'average' 'away' 'awesome'
 'back' 'bad' 'ball' 'ban' 'bank' 'banned' 'based' 'basic' 'basically'
 'became' 'become' 'becomes' 'becoming' 'beef' 'behind' 'belief' 'believe'
 'benefit' 'best' 'best overall' 'best overall post'
 'best overall submitter' 'better' 'bhai' 'bhakts' 'bharat' 'bhi' 'big'
 'bigger' 'biggest' 'bill' 'billion' 'bit' 'bjp' 'black' 'black money'
 'blame' 'block'

In [24]:
smote = SMOTE(random_state=42)

X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

SMOTE is synthetic minority oversampling technique

In [25]:
y_train.value_counts()

category
-1    12533
 1    12533
 0    12533
Name: count, dtype: int64

We can see the class size of all the class became same now.

oversampling step is giving the best result